<a id="0"></a> <br>
# Index

1. [Load the data](#7)
1. [Preprocessing the data](#2)
1. [Cluster Analysis](#6)
    1. [Hierarchical Clustering](#8)
    1. [Partitional Clustering](#9)
    1. [DBSCAN](#10)
    1. [Gaussian Mixture Model](#11)

# 1. Load the data

In [ ]:
import pandas as pd

data = pd.read_csv('data.csv')
data.head()

In [ ]:
data.info()

In [ ]:
summary_statistics = data.describe()
summary_statistics

In [ ]:
column_names = data.columns
column_names

In [ ]:
print(f"Number of duplicate rows: {data.duplicated().sum()}")

In [ ]:
print(data.isnull().sum().value_counts())

In [ ]:
# Display the categorical columns in the data, to know what we need to convert
categorical_data = data.select_dtypes(include=['object'])
print(categorical_data)

In [ ]:
# Count unique values for each specified object-type column and display the counts
object_columns = data[["U95019_s_at","HG3729-HT3999_f_at"]]
for column in object_columns.columns:
    value_counts = object_columns[column].value_counts()
    print(f"\nValue counts for '{column}':")
    print(value_counts)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=data, x='U95019_s_at')
plt.show()
sns.countplot(data=data, x='HG3729-HT3999_f_at')
plt.show()


In [ ]:
import numpy as np
numeric_data = data.select_dtypes(include=np.number)
means = numeric_data.mean(axis=0)
stds = numeric_data.std(axis=0)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
sns.histplot(means, kde=True)
plt.title("Distribution of Means")

plt.subplot(1, 2, 2)
sns.histplot(stds, kde=True)
plt.title("Distribution of Standard Deviations")

plt.show()

# 2. Preprocessing the data

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

categorical_columns = ['U95019_s_at', 'HG3729-HT3999_f_at']
numerical_columns = list(data.drop(categorical_columns, axis=1).columns)

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42) #Split the data before fitting the pipeline

preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', OrdinalEncoder(), categorical_columns),
        ('scaler', RobustScaler(quantile_range=(25.0, 75.0)), numerical_columns)
    ],
    remainder='passthrough'
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

train_processed = pipeline.fit_transform(train_data)
test_processed = pipeline.transform(test_data)

print("Train shape:", train_processed.shape)
print("Test shape:", test_processed.shape)


In [ ]:
train_processed = pipeline.fit_transform(train_data)

ordinal_encoded_data = pipeline.named_steps['preprocessor'].transformers_[0][1].transform(train_data[['U95019_s_at', 'HG3729-HT3999_f_at']])
ordinal_encoded_df = pd.DataFrame(ordinal_encoded_data, columns=['U95019_s_at', 'HG3729-HT3999_f_at'], index=train_data.index)

print(ordinal_encoded_df)


In [ ]:
means = train_processed.mean(axis = 0)
stds = train_processed.std(axis = 0)

plt.figure(figsize = (12, 5))
plt.subplot(1, 2, 1)
sns.histplot(means, kde = True)
plt.title("Distribution of Means")

plt.subplot(1, 2, 2)
sns.histplot(stds, kde = True)
plt.title("Distribution of Standard Deviations")

plt.show()

# 3. Cluster Analysis

<a id="8"></a>
## Hierarchical Clustering

In [ ]:
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

methods = ['single', 'complete', 'average', 'ward']
distance_metrics = ['euclidean', 'cityblock']

best_coefficient = -1
best_combination = {'method': None, 'metric': None}

for method in methods:
    for metric in distance_metrics:
        if method == 'ward' and metric != 'euclidean':
            continue  # Ward only works with Euclidean distance

        distance_matrix = pdist(train_processed, metric=metric)
        linkage_matrix = sch.linkage(distance_matrix, method=method)

        coefficient, _ = cophenet(linkage_matrix, distance_matrix)

        if coefficient > best_coefficient:
            best_coefficient = coefficient
            best_combination = {'method': method, 'metric': metric}

        plt.figure(figsize=(12, 6))
        sch.dendrogram(linkage_matrix)
        plt.title(f"Method: {method}, Metric: {metric}, Cophenetic Coeff: {round(coefficient, 3)}")
        plt.xlabel("Samples")
        plt.ylabel("Distance")
        plt.show()

print("\nBest Combination:")
print(f"Method: {best_combination['method']}")
print(f"Metric: {best_combination['metric']}")
print(f"Cophenetic Coefficient: {round(best_coefficient, 3)}")


In [ ]:
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import fcluster

best_k = 0
best_silhouette = -1

for k in range(2, 11):
    clusters = fcluster(linkage_matrix, t=k, criterion='maxclust')

    silhouette_avg = silhouette_score(train_processed, clusters, metric='euclidean')

    if silhouette_avg > best_silhouette:
        best_silhouette = silhouette_avg
        best_k = k

    print(f"Number of clusters: {k}, Silhouette Score: {round(silhouette_avg, 3)}")

print(f"\nBest number of clusters: {best_k} with silhouette score: {round(best_silhouette, 3)}")


In [ ]:
methods = ['single', 'complete', 'average', 'ward']
distance_metrics = ['euclidean', 'cityblock']

best_coefficient = -1
best_combination = {'method': None, 'metric': None}

for method in methods:
    for metric in distance_metrics:
        if method == 'ward' and metric != 'euclidean':
            continue

        distance_matrix = pdist(test_processed, metric=metric)
        linkage_matrix = sch.linkage(distance_matrix, method=method)

        coefficient, _ = cophenet(linkage_matrix, distance_matrix)

        if coefficient > best_coefficient:
            best_coefficient = coefficient
            best_combination = {'method': method, 'metric': metric}

distance_matrix = pdist(test_processed, metric=best_combination['metric'])
linkage_matrix = sch.linkage(distance_matrix, method=best_combination['method'])

best_silhouette_score = -1
best_num_clusters = 0

for num_clusters in range(2, 11):
    clusters = sch.fcluster(linkage_matrix, num_clusters, criterion='maxclust')

    silhouette_avg = silhouette_score(test_processed, clusters, metric=best_combination['metric'])

    if silhouette_avg > best_silhouette_score:
        best_silhouette_score = silhouette_avg
        best_num_clusters = num_clusters

clusters = sch.fcluster(linkage_matrix, best_num_clusters, criterion='maxclust')

silhouette_avg = silhouette_score(test_processed, clusters, metric=best_combination['metric'])

plt.figure(figsize=(12, 6))
sch.dendrogram(linkage_matrix)
plt.title(f"Best Dendrogram - Method: {best_combination['method']}, Metric: {best_combination['metric']}, "
          f"Cophenetic Coeff: {round(best_coefficient, 3)}, Best Silhouette Score: {round(best_silhouette_score, 3)}")
plt.xlabel("Samples")
plt.ylabel("Distance")
plt.show()

print("\nBest Combination for Test Data:")
print(f"Method: {best_combination['method']}")
print(f"Metric: {best_combination['metric']}")
print(f"Cophenetic Coefficient: {round(best_coefficient, 3)}")
print(f"Best Number of Clusters: {best_num_clusters}")
print(f"Best Silhouette Score: {round(best_silhouette_score, 3)}")



## Partitional Clustering

In [ ]:
from sklearn.cluster import KMeans

silhouette_scores = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=0, n_init=10)
    labels = kmeans.fit_predict(train_processed)
    score = silhouette_score(train_processed, labels)
    silhouette_scores.append(score)
    print(f'K: {k}, Silhouette Score: {score}')


plt.plot(range(2, 10), silhouette_scores, marker='o')
plt.title('Silhouette Scores for different K values')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Silhouette Score')
plt.show()


In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(train_processed)

kmeans_labels = kmeans.labels_
centroids = kmeans.cluster_centers_

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)

plt.scatter(train_processed[:, 5], train_processed[:, 9], c=kmeans_labels, cmap='viridis', alpha=0.6)
plt.scatter(centroids[:, 5], centroids[:, 9], c='red', s=200, alpha=0.8, marker='X')
plt.title('K-means Clustering model ')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

silhouette_avg = silhouette_score(train_processed, kmeans_labels)
print(f'Silhouette Score: {silhouette_avg}')

plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(test_processed)

kmeans_labels = kmeans.labels_
centroids = kmeans.cluster_centers_

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)

plt.scatter(test_processed[:, 5], test_processed[:, 9], c=kmeans_labels, cmap='viridis', alpha=0.6)
plt.scatter(centroids[:, 5], centroids[:, 9], c='red', s=200, alpha=0.8, marker='X')
plt.title('Test data prediction for K-means clustering')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

silhouette_avg = silhouette_score(test_processed, kmeans_labels)
print(f'Silhouette Score: {silhouette_avg}')

plt.show()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 20)
train_pca = pca.fit_transform(train_processed)


In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(train_pca)

kmeans_labels = kmeans.labels_
centroids = kmeans.cluster_centers_

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(train_pca[:, 0], train_pca[:, 1], c=kmeans_labels, cmap='viridis', alpha=0.6)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=200, alpha=0.8, marker='X')
plt.title('K-means Clustering model')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

silhouette_avg = silhouette_score(train_pca, kmeans_labels)
print(f'Silhouette Score: {silhouette_avg}')

plt.show()

## DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

eps_values = np.arange(0.5, 2.5, 5)
min_samples_values = [3, 5, 10, 15, 20]

best_score = -1
best_params = {}

for eps in eps_values:
    for min_samples in min_samples_values:

        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(train_processed)

        unique_labels = set(labels)
        if len(unique_labels) > 1:
            score = silhouette_score(train_processed, labels)
            print(f"eps: {eps:.2f}, min_samples: {min_samples}, Silhouette Score: {score:.4f}")

            if score > best_score:
                best_score = score
                best_params = {'eps': eps, 'min_samples': min_samples}

if best_params:
    print("\nBest Parameters:", best_params)
    print("Best Silhouette Score:", best_score)

    best_dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
    best_labels = best_dbscan.fit_predict(train_processed)
else:
    print("No valid clusters found during grid search.")
    best_labels = np.zeros(train_processed.shape[0], dtype=int)

plt.figure(figsize=(8, 6))
plt.scatter(train_processed[:, 5], train_processed[:, 9], c=best_labels, cmap='viridis', alpha=0.6)
plt.title("DBSCAN Clustering")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()


In [ ]:
data = train_pca

dbscan = DBSCAN(eps=15, min_samples=100)
labels = dbscan.fit_predict(data)

eps_values = np.arange(0.5, 2.5, 5)
min_samples_values = [3, 5, 10, 15, 20]

best_score = -1
best_params = {}

for eps in eps_values:
    for min_samples in min_samples_values:

        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(data)

        unique_labels = set(labels)
        if len(unique_labels) > 1:
            score = silhouette_score(data, labels)
            print(f"eps: {eps:.2f}, min_samples: {min_samples}, Silhouette Score: {score:.4f}")

            if score > best_score:
                best_score = score
                best_params = {'eps': eps, 'min_samples': min_samples}

if best_params:
    print("\nBest Parameters:", best_params)
    print("Best Silhouette Score:", best_score)

    best_dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
    best_labels = best_dbscan.fit_predict(data)
else:
    print("No valid clusters found during grid search.")
    best_labels = np.zeros(data.shape[0], dtype=int)

plt.figure(figsize=(8, 6))
plt.scatter(data[:, 0], data[:, 1], c=best_labels, cmap='viridis', alpha=0.6)
plt.title("DBSCAN Clustering")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()


## Gaussian Mixture Model



In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import ParameterGrid

param_grid = {
    'n_components': [2, 3, 4, 5, 6, 7],
    'covariance_type': ['full', 'tied', 'diag', 'spherical'],
}

best_bic = np.inf
best_params = None
best_gmm = None
best_labels = None

for params in ParameterGrid(param_grid):

    gmm = GaussianMixture(**params, random_state=42)
    gmm.fit(train_processed)
    bic = gmm.bic(train_processed)

    labels = gmm.predict(train_processed)

    if bic < best_bic:
        best_bic = bic
        best_params = params
        best_gmm = gmm
        best_labels = labels

    print(f"n_components: {params['n_components']}, covariance_type: {params['covariance_type']}, BIC: {bic}")

print(f"\nBest Parameters: {best_params}")
print(f"Best BIC: {best_bic}")


best_labels = best_gmm.predict(train_processed)
means = best_gmm.means_

plt.figure(figsize=(8, 6))
plt.scatter(train_processed[:, 5],train_processed[:, 9], c=best_labels, cmap='viridis', alpha=0.6)
plt.title(f'GMM Clustering with {best_params["n_components"]} Components')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()

In [ ]:
test_labels = best_gmm.predict(test_processed)
bic_test = best_gmm.bic(test_processed)
print(f"BIC for Test Data: {bic_test}")
silhouette_test = silhouette_score(test_processed, test_labels)
print(f"Silhouette Score for Test Data: {silhouette_test}")
print(f"\nBest Parameters: {best_params}")

plt.figure(figsize=(8, 6))

plt.scatter(test_processed[:, 5], test_processed[:, 9], c=test_labels, cmap='viridis', alpha=0.6)
plt.title(f'GMM Clustering with {best_params["n_components"]} Components')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()


In [ ]:
param_grid = {
    'covariance_type': ['full', 'tied', 'diag', 'spherical'],
}

best_bic = np.inf
best_params = None
best_gmm = None
best_labels = None

for params in ParameterGrid(param_grid):
    gmm = GaussianMixture(**params, random_state=42)
    gmm.fit(train_pca)
    bic = gmm.bic(train_pca)

    if bic < best_bic:
        best_bic = bic
        best_params = params
        best_gmm = gmm
        best_labels = gmm.predict(train_pca)

print(f"\nBest Parameters: {best_params}")
print(f"Best BIC: {best_bic}")

plt.figure(figsize=(8, 6))
plt.scatter(train_pca[:, 0], train_pca[:, 1], c=best_labels, cmap='viridis', alpha=0.6)
plt.title(f'GMM Clustering with {best_params["n_components"]} Components')
plt.xlabel('PCA Feature 1')
plt.ylabel('PCA Feature 2')
plt.colorbar(label='Cluster Label')
plt.show()
